### 6. 서울시 특정 행정동 치킨외식업체 데이터수집

In [2]:
import re
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup     
from selenium import webdriver
import time
import pandas as pd

# warning ignore
import warnings
warnings.filterwarnings(action='ignore') 
# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
# 검색 지역 받기
keyword = input("검색 동 : ")
print(keyword + '치킨집 103개(정확도 순) 데이터 수집')
print()
print(keyword, '치킨집(카카오맵) 데이터 수집 시작')
# driver 실행
url="https://map.kakao.com/"
path = "./data/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get(url)
time.sleep(0.7)

# 검색창 찾고 키워드 입력
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
searchbox.send_keys(keyword + " 치킨")

# 검색버튼 click
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

# delay 2 sec
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 데이터 프레임 생성
df = pd.DataFrame(columns=['업종','가게이름','평가점수','평가인원', 
                           '리뷰갯수', '주소', "전화", "영업시간","링크주소"])

# 페이지 데이터 추출
crawdata= soup.find_all("li","PlaceItem clickArea")

# 첫번째 페이지의 각 업체 데이터 파싱
for company in range(len(crawdata)):

    li= soup.find_all("li","PlaceItem clickArea")[company]
    upjong = li.find("span","subcategory clickable").text #업종
    shop = li.find("a","link_name").text               #가게이름
    point = li.find("em","num").text                   #평가점수
    person = li.find("a","numberofscore").text[:-1]  #몇명평가
    reviews2 = li.find("a","review").text[3:]         #리뷰-블로그
    add1 = li.find("div","addr")
    add = add1.find_all("p")[0].text                     #주소
    phone1 = li.find("div","contact clickArea")
    phone = phone1.find("span").text                     #전화번호
    selltime = li.find("div","openhour").text.replace("\n","")[3:] #영업시간    
    b_link = li.find_all("a")[7].get('href')    # 해당 업체 상세페이지 링크

    # 데이터 프레임에 저장
    df=df.append({'업종':upjong,'가게이름':shop,'평가점수':point,'평가인원':person, 
                  '리뷰갯수':reviews2, '주소':add, "전화":phone, "영업시간":selltime,"링크주소":b_link}, ignore_index=True)

print(keyword+" 1 번째 페이지 완료")

# 더보기 버튼이 있을경우 더보기 버튼 누르기 (2번)
try:
    time.sleep(1)
    element1 = driver.find_element_by_id("info.search.place.more")
    ActionChains(driver).move_to_element(element1).click(element1).perform()
    ActionChains(driver).move_to_element(element1).click(element1).perform()

    try:
        for nextpage in range(3,6,1):
            time.sleep(1)
            # 2~5번 페이지 데이터 추출
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            crawdata= soup.find_all("li","PlaceItem clickArea")[1]

            # 2~5번 페이지의 각 업체 데이터 파싱 
            for company in range(len(crawdata)):
                selltime=""
                li= soup.find_all("li","PlaceItem clickArea")[company]
                upjong = li.find("span","subcategory clickable").text #업종
                shop = li.find("a","link_name").text               #가게이름
                point = li.find("em","num").text                   #평가점수
                person = li.find("a","numberofscore").text[:-1]      #몇명평가
                reviews2 = li.find("a","review").text[3:]           #리뷰-블로그
                add1 = li.find("div","addr")
                add = add1.find_all("p")[0].text                     #주소
                phone1 = li.find("div","contact clickArea")
                phone = phone1.find("span").text                     #전화번호
                selltime = li.find("div","openhour").text.replace("\n","")[3:] #영업시간
                b_link = li.find_all("a")[7].get('href')                # 해당 업체 상세페이지 링크

                # 데이터프레임에 저장 붙히기
                df=df.append({'업종':upjong,'가게이름':shop,'평가점수':point,'평가인원':person, 
                              '리뷰갯수':reviews2, '주소':add, "전화":phone, "영업시간":selltime,"링크주소":b_link}, ignore_index=True)

            print( keyword, nextpage-1, "번째 페이지 완료")

            time.sleep(1)

            # 다음 페이지 버튼 누르기
            element1 = driver.find_element_by_id("info.search.page.no{}".format(nextpage))
            ActionChains(driver).move_to_element(element1).click(element1).perform()
    except:
        pass
    print(keyword+" 5 번째 페이지 완료")    

    time.sleep(1)
    # 5번 페이지 이후의 다음 목록 페이지 화살표 누르기
    element1 = driver.find_element_by_id("info.search.page.next")
    ActionChains(driver).move_to_element(element1).click(element1).perform()


    for nextpage in range(2,7,1):
        # 6~10번 페이지 데이터 추출
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        crawdata= soup.find_all("li","PlaceItem clickArea")[1]
        # 6~10번 페이지의 각 업체 데이터 파싱
        for company in range(len(crawdata)):
            selltime=""
            li= soup.find_all("li","PlaceItem clickArea")[company]
            upjong = li.find("span","subcategory clickable").text #업종
            shop = li.find("a","link_name").text               #가게이름
            point = li.find("em","num").text                   #평가점수
            person = li.find("a","numberofscore").text[:-1]    #몇명평가
            reviews2 = li.find("a","review").text[3:]          #리뷰-블로그
            add1 = li.find("div","addr")
            add = add1.find_all("p")[0].text                     #주소
            phone1 = li.find("div","contact clickArea")
            phone = phone1.find("span").text                     #전화번호
            selltime = li.find("div","openhour").text.replace("\n","")[3:] #영업시간
            b_link = li.find_all("a")[7].get('href') # 해당 업체 상세페이지 링크

            # 데이터 프레임에 저장 붙히기
            df=df.append({'업종':upjong,'가게이름':shop,'평가점수':point,'평가인원':person, 
                          '리뷰갯수':reviews2, '주소':add, "전화":phone, "영업시간":selltime,"링크주소":b_link}, ignore_index=True)

        print( keyword, nextpage+4, "번째 페이지 완료")
        time.sleep(1)

        # 다음 페이지 버튼 누르기
        element1 = driver.find_element_by_id("info.search.page.no{}".format(nextpage))
        ActionChains(driver).move_to_element(element1).click(element1).perform()

except:
    pass

# 저장 위치 디렉토리 미리 생성되어있어야함
# csv로 각 지역별로 저장
savepath = "./data/"
df.to_csv(savepath+keyword+" 치킨집 카카오맵 데이터 크롤링.csv", index = False, mode='w', encoding = 'utf-8-sig')
print(keyword+' 완료'+'================')
print()
driver.close()
    
print()
print('전체 데이터 수집 완료')

검색 동 : 묵1동
묵1동치킨집 103개(정확도 순) 데이터 수집

묵1동 치킨집(카카오맵) 데이터 수집 시작
묵1동 1 번째 페이지 완료
묵1동 2 번째 페이지 완료
묵1동 3 번째 페이지 완료
묵1동 4 번째 페이지 완료
묵1동 5 번째 페이지 완료
묵1동 6 번째 페이지 완료
묵1동 7 번째 페이지 완료
묵1동 8 번째 페이지 완료
묵1동 9 번째 페이지 완료
묵1동 10 번째 페이지 완료
묵1동 완료================


전체 데이터 수집 완료
